# Web Scraping de Casas en Venta en Zona 10

## Descripción
Este proyecto realiza **web scraping** en la página de Mapainmueble para extraer información detallada de las propiedades en venta en la **Zona 10, Guatemala**.

### Importamos Librerias

- requests
- BS4
- pandas


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

print("Librerías instaladas correctamente")


Librerías instaladas correctamente


### Status
- Revisamos el estatus de la pagina y que realmente estamos tratando con HTML para hacer el scrapping estatico con BS4.

In [2]:
import requests

URL = "https://mapainmueble.com/casas-en-venta-zona-10/"

print(f"Status Code: {response.status_code}")


Status Code: 200


### Scrappping code


In [41]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re  # Para limpieza de datos

# Base URL para la paginación
BASE_URL = "https://mapainmueble.com/casas-en-venta-zona-10/"
headers = {"User-Agent": "Mozilla/5.0"}

data = []  # Lista para almacenar todas las propiedades

# Iterar sobre las páginas (ajustar si hay más de 3 páginas)
for page in range(1, 4):  
    print(f"Scrapeando página {page}...")
    url = f"{BASE_URL}page/{page}/" if page > 1 else BASE_URL
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        propiedades = soup.find_all("div", class_="property_listing property_card_default")

        for prop in propiedades:
            try:
                titulo = prop.find("a").text.strip()
            except AttributeError:
                titulo = None

            # Extraer precio correctamente
            try:
                precio = prop.find("div", class_="listing_unit_price_wrapper").text.strip().split("\n")[0]
            except AttributeError:
                precio = None

            try:
                ubicacion = "Zona 10, Guatemala"
            except AttributeError:
                ubicacion = None

            try:
                habitaciones = prop.find("span", class_="inforoom").text.strip()
            except AttributeError:
                habitaciones = None

            try:
                baños = prop.find("span", class_="infobath").text.strip()
            except AttributeError:
                baños = None

            try:
                metros_cuadrados = prop.find("span", class_="infosize").text.strip().split(" ")[0]
            except AttributeError:
                metros_cuadrados = None

            try:
                url_propiedad = prop.find("a")["href"]
            except AttributeError:
                url_propiedad = None

            try:
                inmobiliaria = prop.find("div", class_="property_agent_wrapper").find("a").text.strip()
            except AttributeError:
                inmobiliaria = None

            # Agregar datos a la lista
            data.append([titulo, precio, ubicacion, habitaciones, baños, metros_cuadrados, inmobiliaria, url_propiedad])

    else:
        print(f"Error al acceder a la página {page}: {response.status_code}")

# Convertir a DataFrame
df = pd.DataFrame(data, columns=["Título", "Precio", "Ubicación", "Habitaciones", "Baños", "Metros Cuadrados", "Inmobiliaria", "URL"])

# Guardar en CSV
df.to_csv("casas_zona_10_final.csv", index=False)

# Mostrar las primeras filas del DataFrame
print(df.head())


Scrapeando página 1...
Scrapeando página 2...
Scrapeando página 3...
    Título                   Precio           Ubicación Habitaciones Baños  \
0  Zona 10  $ 850,000 Más impuestos  Zona 10, Guatemala            4   4.5   
1  Zona 10    $ 420,000 negociables  Zona 10, Guatemala            4   3.5   
2  Zona 10              $ 1,200,000  Zona 10, Guatemala            4   2.5   
3  Zona 10                $ 330,000  Zona 10, Guatemala            3   2.5   
4  Zona 10                $ 980,000  Zona 10, Guatemala            4   4.5   

  Metros Cuadrados                   Inmobiliaria  \
0              482             GIEKA INMOBILIARIA   
1              250  Sarissa Asesoria Inmobiliaria   
2              363            Juan Carlos Ramirez   
3              200          Ambiente Inmobiliario   
4             None                       M2 Guate   

                                      URL  
0  https://mapainmueble.com/area/zona-10/  
1  https://mapainmueble.com/area/zona-10/  
2  https://

### Nuevo Script para jalar unicmanete los datos numericos que oucpamos para nuestro modelo


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re  # Para limpieza de datos

# Base URL para la paginación
BASE_URL = "https://mapainmueble.com/casas-en-venta-zona-10/"
headers = {"User-Agent": "Mozilla/5.0"}

data = []  # Lista para almacenar todas las propiedades

# Iterar sobre las páginas (hasta 9 páginas como viste en la paginación)
for page in range(1, 10):  
    print(f"Scrapeando página {page}...")
    url = f"{BASE_URL}page/{page}/" if page > 1 else BASE_URL
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        propiedades = soup.find_all("div", class_="property_listing property_card_default")

        for prop in propiedades:
            # Extraer precio correctamente y limpiarlo
            try:
                precio_texto = prop.find("div", class_="listing_unit_price_wrapper").text.strip()
                precio = re.search(r"([\d,]+)", precio_texto)
                precio = precio.group(1).replace(",", "") if precio else None
                precio = int(precio) if precio else None  # Convertir a entero
            except AttributeError:
                precio = None

            # Extraer número de habitaciones
            try:
                habitaciones = prop.find("span", class_="inforoom").text.strip()
                habitaciones = int(re.search(r"(\d+)", habitaciones).group(1))
            except (AttributeError, ValueError, TypeError):
                habitaciones = None

            # Extraer número de baños
            try:
                baños = prop.find("span", class_="infobath").text.strip()
                baños = float(re.search(r"(\d+(\.\d+)?)", baños).group(1))
            except (AttributeError, ValueError, TypeError):
                baños = None

            # Extraer metraje en metros cuadrados
            try:
                metros_cuadrados = prop.find("span", class_="infosize").text.strip().split(" ")[0]
                metros_cuadrados = int(re.sub(r"[^\d]", "", metros_cuadrados))  # Extrae solo números
            except (AttributeError, ValueError, TypeError):
                metros_cuadrados = None

            # Agregar datos a la lista SOLO si no hay valores nulos
            if None not in [precio, habitaciones, baños, metros_cuadrados]:
                data.append([precio, habitaciones, baños, metros_cuadrados])

    else:
        print(f"Error al acceder a la página {page}: {response.status_code}")

# Convertir a DataFrame
df = pd.DataFrame(data, columns=["Precio", "Habitaciones", "Baños", "Metraje (m2)"])

# Guardar en CSV solo si tiene datos
if not df.empty:
    df.to_csv("casas_zona_10_limpio.csv", index=False)
    print("\n Archivo 'casas_zona_10_limpio.csv' guardado correctamente!")
else:
    print("\n No se extrajeron datos válidos. Revisa el scraping.")

# Mostrar las primeras filas del DataFrame
print(df.head())
print(f"\n Total de propiedades extraídas: {len(df)}")


Scrapeando página 1...
Scrapeando página 2...
Scrapeando página 3...
Scrapeando página 4...
Scrapeando página 5...
Scrapeando página 6...
Scrapeando página 7...
Scrapeando página 8...
Scrapeando página 9...

✅ Archivo 'casas_zona_10_limpio.csv' guardado correctamente!
    Precio  Habitaciones  Baños  Metraje (m2)
0   229487             3    2.5           240
1   900000             3    2.5           400
2  1050000             4    4.5           408
3   980000             4    4.5           457
4   850000             4    4.5           482

📊 Total de propiedades extraídas: 65
